# **ASHRAE Energy Prediction**

In [ ]:
# Import Statements

import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import statistics
import seaborn as sns
from lightgbm import LGBMRegressor, plot_importance
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from copy import copy
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # Any results you write to the current directory are saved as output.
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)

In [ ]:
# Code from https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction 
# Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# function to calculate evaluation metric
def rmsle(y_true: pd.Series, y_predict: pd.Series) -> float:
    """
    Evaluate root mean squared log error
    :param y_true:
    :param y_predict:
    :return:
    """
    return np.sqrt(msle(y_true, y_predict))

In [ ]:
# Import data
INPUT = "../input/ashrae-energy-prediction/"

df_train = pd.read_csv(f"{INPUT}train.csv")
df_test = pd.read_csv(f"{INPUT}test.csv")
bldg_metadata = pd.read_csv(f"{INPUT}building_metadata.csv")
weather_train = pd.read_csv(f"{INPUT}weather_train.csv")
weather_test = pd.read_csv(f"{INPUT}weather_test.csv")
sample = pd.read_csv(f"{INPUT}sample_submission.csv")

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_test = df_test.drop(columns=['row_id'])

In [ ]:
df_train = reduce_mem_usage(df=df_train)
df_test = reduce_mem_usage(df=df_test)
weather_train = reduce_mem_usage(df=weather_train)
weather_test = reduce_mem_usage(df=weather_test)

In [ ]:
df_train = df_train.merge(bldg_metadata, on='building_id', how='left')
df_test = df_test.merge(bldg_metadata, on='building_id', how='left')
df_train = df_train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
df_test = df_test.merge(weather_test, on=['site_id', 'timestamp'], how='left')

In [ ]:
import gc
del weather_train, weather_test, bldg_metadata
gc.collect()

In [ ]:
df_train['timestamp'] = pd.to_datetime(arg=df_train['timestamp'])
df_test['timestamp'] = pd.to_datetime(arg=df_test['timestamp'])

## Feature Engineering

In [ ]:
# Extracting date features from timestamp
df_train['year'] = df_train['timestamp'].dt.year
df_train['month'] = df_train['timestamp'].dt.month
df_train['day'] = df_train['timestamp'].dt.day
df_train['hour'] = df_train['timestamp'].dt.hour
df_test['year'] = df_test['timestamp'].dt.year
df_test['month'] = df_test['timestamp'].dt.month
df_test['day'] = df_test['timestamp'].dt.day
df_test['hour'] = df_test['timestamp'].dt.hour
df_train['dayofweek'] = df_train['timestamp'].dt.dayofweek
df_test['dayofweek'] = df_test['timestamp'].dt.dayofweek

# 1: day, 2:night. Hope this save some memory
df_train['day-and-night'] = np.where((df_train['hour'] < 6), 2, 1)
df_test['day-and-night'] = np.where((df_test['hour'] < 6), 2, 1)

#1: winter, 2: spring, 3: summer, 4: autumn. Hope this save some memory 
df_train.loc[df_train['month'].isin([12, 1, 2]), 'season'] = 1
df_train.loc[df_train['month'].isin([3, 4, 5]), 'season'] = 2
df_train.loc[df_train['month'].isin([6, 7, 8]), 'season'] = 3
df_train.loc[df_train['month'].isin([9, 10, 11]), 'season'] = 4
df_test.loc[df_test['month'].isin([12, 1, 2]), 'season'] = 1
df_test.loc[df_test['month'].isin([3, 4, 5]), 'season'] = 2
df_test.loc[df_test['month'].isin([6, 7, 8]), 'season'] = 3
df_test.loc[df_test['month'].isin([9, 10, 11]), 'season'] = 4

In [ ]:
# Because we made few new features we can try to reduce memory once again
df_train = reduce_mem_usage(df=df_train)
df_test = reduce_mem_usage(df=df_test)

In [ ]:
# Making age feature
df_train['age'] = df_train['year'] - df_train['year_built']
df_test['age'] = df_test['year'] - df_test['year_built']

# Making number of hours passed from start
new_df = df_train.groupby(by=['building_id'], as_index=False)['timestamp'].min()
new_df = new_df.rename(columns = {'timestamp': 'start_ts'})

df_train = df_train.merge(new_df, on = 'building_id', how='left')
df_test = df_test.merge(new_df, on = 'building_id', how='left')

df_train['hours_passed'] = (df_train['timestamp'] - df_train['start_ts']).dt.total_seconds()/3600
df_test['hours_passed'] = (df_test['timestamp'] - df_test['start_ts']).dt.total_seconds()/3600

In [ ]:
# Because we made few new features we can try to reduce memory once again
df_train = reduce_mem_usage(df=df_train)
df_test = reduce_mem_usage(df=df_test)

In [ ]:
# Making combination of categorical variable to see if they help model become better
# df_train['building_id_meter_hours_passed'] = df_train['building_id'].astype(str) + '_' + df_train['meter'].astype(str) + '_' + df_train['hours_passed'].astype(str)
# df_test['building_id_meter_hours_passed'] = df_test['building_id'].astype(str) + '_' + df_test['meter'].astype(str) + '_' + df_test['hours_passed'].astype(str)

# # Because we made few new features we can try to reduce memory once again
# df_train = reduce_mem_usage(df=df_train)
# df_test = reduce_mem_usage(df=df_test)

# df_train['building_id_meter'] = df_train['building_id'].astype(str) + '_' + df_train['meter'].astype(str)
# df_test['building_id_meter'] = df_test['building_id'].astype(str) + '_' + df_test['meter'].astype(str)

## Data Cleaning

In [ ]:
# site_id =0 has some building where meter readings before May 21, 2016 are not reliable so dropping those records 
df_train = df_train.query('not(site_id==0 & timestamp<"2016-05-21 00:00:00")')

df_train = df_train.loc[df_train['meter_reading'] > 0, :]
# df_test = df_test.loc[df_test['meter_reading'] > 0, :]

# Missing value handling
cols = ['floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 
        'wind_direction', 'wind_speed']
df_train.loc[:, cols] = df_train.loc[:, cols].interpolate(axis=0)
df_test.loc[:, cols] = df_test.loc[:, cols].interpolate(axis=0)

In [ ]:
# Convert to categorical datatype
cat_cols = ['meter', 'primary_use', 'site_id', 'building_id', 'year', 'month', 'day', 'hour', 'dayofweek', 'season', 'day-and-night']
for col in cat_cols:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')

In [ ]:
df_train.head()

## Modelling

In [ ]:
df_train = df_train.reset_index(drop=True)

In [ ]:
# Make validation set based on time split
df_val = df_train.loc[df_train['timestamp'] >= '2016-11-01 00:00:00']
df_train = df_train.loc[df_train['timestamp'] < '2016-11-01 00:00:00']
y_train = df_train['meter_reading']
y_val = df_val['meter_reading']

y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
df_train = df_train.drop(columns=['meter_reading'])
df_val = df_val.drop(columns=['meter_reading'])

# # Create input and target
# y_train = df_train['meter_reading']
# y_train = np.log1p(y_train)
# df_train = df_train.drop(columns=['meter_reading'])

# # Make validation set based on train_test_split
# df_train, df_val, y_train, y_val = train_test_split(df_train, y_train, test_size=0.2, random_state=42)

# Drop timestamp because model does not accept
df_train = df_train.drop(columns=['timestamp', 'start_ts'])
df_val = df_val.drop(columns=['timestamp', 'start_ts'])
df_test = df_test.drop(columns=['timestamp', 'start_ts'])

In [ ]:
# # Cross Validation
# scores = []
# tss = TimeSeriesSplit(n_splits=5)
# fold = 0
# for train_index, val_index in tss.split(df_train): 
#     fold+=1
#     lgbmr = LGBMRegressor(n_estimators=1000, random_state=10)
#     lgbmr.fit(df_train.loc[train_index, :], y_train[train_index])
#     y_predict = lgbmr.predict(df_train.loc[val_index, :])
#     score = np.sqrt(mse(y_train[val_index], y_predict))
#     print(f"fold{fold}: {score}")
#     scores.append(score)
# print(f"Mean score: {sum(scores)/len(scores)}    Std. dev: {statistics.stdev(scores)}")

In [ ]:
gc.collect()

In [ ]:
# Model
lgbmr = LGBMRegressor(n_estimators=500, random_state=10)
lgbmr.fit(df_train, y_train)
y_predict = lgbmr.predict(df_val)
score = np.sqrt(mse(y_val, y_predict))
# score = rmsle(y_val, y_predict)
print(f"score: {score}")

# # Training the model on full train dataset
# lgbmr.fit(pd.concat([df_train, df_val], axis=0), pd.concat([y_train, y_val], axis=0))

In [ ]:
# Lets try to visualize model predicts vs actual meter_readings
viz_data = pd.concat(objs=[df_val, y_val, pd.Series(data=y_predict, name='predictions', index=df_val.index)], 
                     axis=1)

viz_data['error'] = np.abs(y_predict - y_val)

lgbmr_errors = (viz_data.groupby(by=['site_id', 'building_id', 'meter'], as_index=False, observed=True)['error'].mean()).merge(df_val.loc[:, ['site_id', 'building_id', 'meter', 'primary_use']].drop_duplicates(), on = ['site_id', 'building_id', 'meter'])

In [ ]:
lgbmr_errors.sort_values(by='error', ascending=False).head()

In [ ]:
# # lets viualize what is going on wrong with these buildings in train and test set both

# #Train set
# fig, ax = plt.subplots(figsize=(12,9))
# data = df_train.copy()
# data['meter_reading'] = y_train
# data = data.loc[(data['site_id'] == 7) & (data['building_id'] == 799) & (data['meter'] == 0), :].reset_index(drop=True)
# ax.scatter(data.index ,data['meter_reading'], c='blue', s=5)
# ax.legend(data['primary_use'])
# plt.show()

In [ ]:
# lets viualize what is going on wrong with these buildings in train and test set both

#Train set
for row in lgbmr_errors.sort_values(by='error', ascending=False).head(10).index:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharex=True, sharey=True)
    data = df_train.copy()
    data['meter_reading'] = y_train
    data = data.loc[(data['site_id'] == lgbmr_errors.loc[row, 'site_id']) & (data['building_id'] == lgbmr_errors.loc[row, 'building_id']) & (data['meter'] == lgbmr_errors.loc[row, 'meter']), :].reset_index(drop=True)
    ax1.scatter(data.index ,data['meter_reading'], c='blue', s=5)
    ax1.set_title(f"{lgbmr_errors.loc[row, 'site_id']}-{lgbmr_errors.loc[row, 'building_id']}-{lgbmr_errors.loc[row, 'meter']}")
    ax1.legend(data['primary_use'])
    data = viz_data.loc[(viz_data['site_id'] == lgbmr_errors.loc[row, 'site_id']) & (viz_data['building_id'] == lgbmr_errors.loc[row, 'building_id']) & (viz_data['meter'] == lgbmr_errors.loc[row, 'meter']), :].reset_index(drop=True)
    ax2.scatter(data.index ,data['meter_reading'], c='blue', s=5)
    ax2.scatter(data.index ,data['predictions'], c='orange', s=5)
    ax2.set_title(f"{lgbmr_errors.loc[row, 'site_id']}-{lgbmr_errors.loc[row, 'building_id']}-{lgbmr_errors.loc[row, 'meter']}")
    ax2.legend(data['primary_use'])
    plt.show()

In [ ]:
plot_importance(lgbmr)

In [ ]:
# #Val set
# fig, ax = plt.subplots(figsize=(12,9))
# data = viz_data.loc[(viz_data['site_id'] == 7) & (viz_data['building_id'] == 799) & (viz_data['meter'] == 0), :].reset_index(drop=True)
# ax.scatter(data.index ,data['meter_reading'], c='blue', s=5)
# ax.scatter(data.index ,data['predictions'], c='orange', s=5)
# ax.legend(data['primary_use'])

## Ensemble Modelling

In [ ]:
# Half and half learning
X_1st_half = df_train[:int(df_train.shape[0]/2)]
y_1st_half = y_train[:int(df_train.shape[0]/2)]
X_2nd_half = df_train[int(df_train.shape[0]/2):]
y_2nd_half = y_train[int(df_train.shape[0]/2):]

In [ ]:
lgbmr_1st_half = LGBMRegressor(random_state=10)
lgbmr_2nd_half = LGBMRegressor(random_state=10)
lgbmr_1st_half.fit(X_1st_half, y_1st_half)
lgbmr_2nd_half.fit(X_2nd_half, y_2nd_half)
y_predict_1 = lgbmr_1st_half.predict(df_val)
y_predict_2 = lgbmr_2nd_half.predict(df_val)
y_predict_1_2 = (pd.Series(data=y_predict_1, name='prediction_1') + pd.Series(data=y_predict_2, name='prediction_2'))/2
score = np.sqrt(mse(y_val, y_predict_1_2))
print(f"score: {score}")

In [ ]:
# Training model on entire dataset using Half and half learning methodology
X_1st_half = df_train[:int(pd.concat([df_train, df_val], axis=0).shape[0]/2)]
y_1st_half = y_train[:int(pd.concat([df_train, df_val], axis=0).shape[0]/2)]
X_2nd_half = df_train[int(pd.concat([df_train, df_val], axis=0).shape[0]/2):]
y_2nd_half = y_train[int(pd.concat([df_train, df_val], axis=0).shape[0]/2):]

lgbmr_1st_half = LGBMRegressor(random_state=10)
lgbmr_2nd_half = LGBMRegressor(random_state=10)
lgbmr_1st_half.fit(X_1st_half, y_1st_half)
lgbmr_2nd_half.fit(X_2nd_half, y_2nd_half)

In [ ]:
# Saving model
filename = 'lgbm_model1.pickle'
pickle.dump(lgbmr, open(filename, 'wb'))
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

filename = 'lgbmr_1st_half.pickle'
pickle.dump(lgbmr, open(filename, 'wb'))
# load the model from disk
lgbmr_1st_half = pickle.load(open(filename, 'rb'))
filename = 'lgbmr_2nd_half.pickle'
pickle.dump(lgbmr, open(filename, 'wb'))
# load the model from disk
lgbmr_2nd_half = pickle.load(open(filename, 'rb'))

In [ ]:
# # Important features
# fig, ax = plt.subplots(figsize=(12, 9))
# plot_importance(lgbmr, ax=ax)

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 9))
# plot_importance(lgbmr_1st_half, ax=ax)

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 9))
# plot_importance(lgbmr_2nd_half, ax=ax)

In [ ]:
del df_train, df_val, y_train, y_val, lgbmr
gc.collect()

In [ ]:
# Make predictions on test set
# STEP = 1000000
# y_test_predict = []
# for i in range(0, df_test.shape[0], STEP):
#     batch_prediction = loaded_model.predict(df_test.loc[i:i+STEP-1,:])
#     y_test_predict.append(list(batch_prediction))
# y_test = []
# for predictions in y_test_predict:
#     y_test = y_test + predictions

STEP = 1000000
y_test_predict = []
for i in range(0, df_test.shape[0], STEP):
    batch_prediction = lgbmr_1st_half.predict(df_test.loc[i:i+STEP-1,:])
    y_test_predict.append(list(batch_prediction))
y_test_1st_half = []
for predictions in y_test_predict:
    y_test_1st_half = y_test_1st_half + predictions
    
STEP = 1000000
y_test_predict = []
for i in range(0, df_test.shape[0], STEP):
    batch_prediction = lgbmr_2nd_half.predict(df_test.loc[i:i+STEP-1,:])
    y_test_predict.append(list(batch_prediction))
y_test_2nd_half = []
for predictions in y_test_predict:
    y_test_2nd_half = y_test_2nd_half + predictions


In [ ]:
# sample['meter_reading'] = y_test
sample['meter_reading'] = (pd.Series(data=y_test_1st_half, name='pred_1st_half') + 
                           pd.Series(data=y_test_2nd_half, name='pred_2nd_half'))/2 

In [ ]:
from math import e
# sample['meter_reading'] = e**sample['meter_reading'] - 1
sample['meter_reading'] = np.expm1(sample['meter_reading'])

In [ ]:
sample.to_csv("submission.csv", index=False, float_format='%.4f')

References:
* https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction
* https://www.kaggle.com/rohanrao/ashrae-half-and-half